## Task 1

The task one is to develop an application to detect and track moving cars from a camera recording using openCV library. 

Install the packages required. We install openCV for computer vision and numpy to handle arrays and matrices.

In [ ]:
# install openCV
!pip install opencv-python
# install numpy
!pip install numpy

Import the libraries

In [1]:
# import openCV library and numpy
import cv2
import numpy as np

Create a video caputre object with the video file.

**Note: Please place the video in the same directory of the Jupyter notebook**

In [2]:
# capture the video
video = cv2.VideoCapture('Traffic_Laramie_1.mp4')

The following code will process the video frames and detect and track cars that are moving on the main street. We use OpenCV's BackgroundSubtractorMOG2, A Gaussian mixture-based background/foreground segmentation algorithm used for background subtraction. Background subtraction can be done by subtracting a static frame from the current frame. But this is an inefficient method. BackgroundSubtractorMOG2 algorithm is one of the efficient methods to detect objects in real-time video.

We use the follwoing steps to detect the cars in the video.

1. Capture the video
2. Read the video frames
3. Resize the frame to half
4. Draw the line to mark the main street
5. Apply grey conversion (noise reduction and smoothening)
6. Apply Gaussian blur
7. Apply foreground mask
8. Apply thresholding
9. Dilation (Morphological operation to increase the object area)
10. Extract and mark the objects using contours
11. Show the video tracking the moving cars

**The code is given below and explained in detail using line comments. Press the button 'q' to close the videos**

In [3]:
# The following code is inspired by the method discussed in analyticsvidhya.com and the lectures 
# No code is copied directly from from the given reference and it is written in my own way.
# [reference 2, Link: https://medium.com/@ggaighernt/optical-flow-and-motion-detection-5154c6ba4419]

# The create background substraction object using BackgroundSubtractorMOG2 class
bgsub_obj = cv2.createBackgroundSubtractorMOG2(detectShadows = True)
# Initialize a matrix of 3x3 size as the kernel for dilation
kernel = np.ones((3,3),np.uint8)

# Start an infinite loop. Press button 'q' to exit
while True:
    

    # Read the video frames using the VideoCapture object
    ret, frame = video.read()
    # ret will be false if no frame is captured
    # check if the frame is grabbed successfully
    if not ret:
        continue
    
    # Resize frame to half. It will make the frame processing by the algorithms much faster
    frame_resized = cv2.resize(frame, (0, 0), None, 0.5, 0.5)
    
    # Draw a line to mark the main street
    cv2.line(frame_resized, (0, 150),(520,150),(255, 0, 0))

    # use cvtColor to convert the frame from one colour space to another
    # Here we convert it to cv2.COLOR_BGR2GRAY colour space (grayscale)
    # Gray conversion reduce dimensions, model complexity, make it compatible for other algorithms
    gray_frame=cv2.cvtColor(frame_resized,cv2.COLOR_BGR2GRAY)
    
    # Gaussian blur: Frame is colvolved with Gaussian filter
    # Removes Gaussian noise and high frequency components
    blur_frame=cv2.GaussianBlur(gray_frame,(5,5),0)
    
    # apply the foreground mask using createBackgroundSubtractorMOG2 object's 'apply' method
    # It will be applied on each frame
    frame_mask = bgsub_obj.apply(blur_frame)
    
    # Thresholding: Each pixel values is compared to the threshold - 150
    # If the pixel value is less than 150 it is set to 0 otherwise 255
    threshold_frame=cv2.threshold(frame_mask,150,255, cv2.THRESH_BINARY)[1]
    
    # Dilation: The kernel matrix is applied to convolve the frame
    # It increases the white region and make the object detectible
    dilated = cv2.dilate(threshold_frame, kernel, iterations = 2)

    # Contours: The findContours() method of openCV helps to find the contours in the threshold_frame
    # It uses simple aproximation method to find the endpoints of the objects
    (contours,_)=cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Iterate over the contours and mark the object
    for c in contours:
        
        # Draw a rectangle around the object
        (x, y, w, h)=cv2.boundingRect(c)
        
        # contourArea() method filters out any small contours ignore it
        # if y value is less than 150, the object is not in the main street
        if cv2.contourArea(c) < 1000 or y < 150:
            continue
            
        # add the rectangle to the resized frame
        cv2.rectangle(frame_resized, (x, y), (x+w, y+h), (0,255,0), 1)
    
    # show the resized and dilated videos
    cv2.imshow('Video', frame_resized)
    cv2.imshow('Dilated', dilated)
    
    # Press 'q' to 'quit'
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# release the video capture object and close all windows
video.release()
cv2.destroyAllWindows()

## Reference

1. https://www.analyticsvidhya.com/blog/2022/03/vehicle-motion-detection-using-background-subtraction/